<a href="https://colab.research.google.com/github/pr0fi7/image_inpainter/blob/main/image_inpainting_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing the dependencies

In [1]:
!pip install opencv-python-headless

In [2]:
!pip install fastapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 833.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [4]:
!pip install diffusers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.2 MB/s eta 0:00:00


In [5]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 7.0 MB/s eta 0:00:00


In [6]:
!pip install nest-asyncio
!pip install python-multipart==0.0.7
!pip install uvicorn==0.29.0
!pip install pyngrok


  Attempting uninstall: python-multipart
    Found existing installation: python-multipart 0.0.9
    Uninstalling python-multipart-0.0.9:
      Successfully uninstalled python-multipart-0.0.9


In [7]:
from fastapi import FastAPI, File, UploadFile, Response
from fastapi.middleware.cors import CORSMiddleware

import cv2
import numpy as np
import json
import PIL
import requests
import torch
from io import BytesIO

from diffusers import StableDiffusionInpaintPipeline


pipeline = StableDiffusionInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting",
    torch_dtype=torch.float16,

)
pipeline = pipeline.to("cuda")


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

unet/diffusion_pytorch_model.safetensors not found


...

tokenizer/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/335M [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [18]:

def transform_image(init_image, mask_image, text):
    prompt = text + ", 8k resolution, detailed, beatiful eyes"
    negative_prompt = 'ugly, distorted, disformed, change face, problem with fingers'
    init_image = PIL.Image.open(init_image).convert("RGB")
    mask_image = PIL.Image.open(mask_image).convert("RGB")
    image = pipeline(prompt=prompt, image=init_image, mask_image=mask_image, height=1024, width=1024).images[0]
    image_np = np.array(image)
    image_np = image_np.astype(np.uint8)
a 
    return image_np


In [9]:


def resize_image(image):
    # Load the input image
    input_image = cv2.imread(image)

    # Get the dimensions of the input image
    original_height, original_width = input_image.shape[:2]

    # Determine the maximum dimension for resizing
    max_dim = max(original_height, original_width)

    # Calculate the scaling factor to resize the image while preserving the aspect ratio
    scale_factor = 1024 / max_dim

    # Resize the image using the scaling factor
    resized_image = cv2.resize(input_image, (0, 0), fx=scale_factor, fy=scale_factor)

    # Get the new dimensions of the resized image
    resized_height, resized_width = resized_image.shape[:2]

    # Calculate the padding needed to make the image 1024x1024
    top_pad = (1024 - resized_height) // 2
    bottom_pad = 1024 - resized_height - top_pad
    left_pad = (1024 - resized_width) // 2
    right_pad = 1024 - resized_width - left_pad

    # Pad the image with black spaces
    resized_image = cv2.copyMakeBorder(resized_image, top_pad, bottom_pad, left_pad, right_pad, cv2.BORDER_CONSTANT, value=(0, 0, 0))

    return resized_image

In [20]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 35.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.3 which is incompatible.


In [23]:
import mediapipe
import cv2
import matplotlib.pyplot as plt
import pandas as pd

def create_mask(resized_image):


    mp_face_mesh = mediapipe.solutions.face_mesh
    face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)

    results = face_mesh.process(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
    landmarks = results.multi_face_landmarks[0]


    face_oval = mp_face_mesh.FACEMESH_FACE_OVAL


    df = pd.DataFrame(list(face_oval), columns = ["p1", "p2"])
    routes_idx = []

    p1 = df.iloc[0]["p1"]
    p2 = df.iloc[0]["p2"]

    for i in range(0, df.shape[0]):

        #print(p1, p2)

        obj = df[df["p1"] == p2]
        p1 = obj["p1"].values[0]
        p2 = obj["p2"].values[0]

        route_idx = []
        route_idx.append(p1)
        route_idx.append(p2)
        routes_idx.append(route_idx)

    # -------------------------------

    # for route_idx in routes_idx:
    #     print(f"Draw a line between {route_idx[0]}th landmark point to {route_idx[1]}th landmark point")
    routes = []

    for source_idx, target_idx in routes_idx:

        source = landmarks.landmark[source_idx]
        target = landmarks.landmark[target_idx]

        relative_source = (int(resized_image.shape[1] * source.x), int(resized_image.shape[0] * source.y))
        relative_target = (int(resized_image.shape[1] * target.x), int(resized_image.shape[0] * target.y))

        #cv2.line(img, relative_source, relative_target, (255, 255, 255), thickness = 2)

        routes.append(relative_source)
        routes.append(relative_target)
    mask = np.zeros((resized_image.shape[0], resized_image.shape[1]))
    mask = cv2.fillConvexPoly(mask, np.array(routes), 255)

    inverted_mask = 255 - mask

    return inverted_mask

    # # Convert the image to grayscale
    # gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

    # # Face detection using a pre-trained model (e.g., Haar cascades or MTCNN)
    # face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    # faces = face_cascade.detectMultiScale(gray_image, minNeighbors=1, minSize=(50, 50))

    # # Initialize an empty mask
    # mask = np.zeros_like(gray_image)

    # # Iterate over detected faces
    # for (x, y, w, h) in faces:
    #     # Calculate the new bounding box dimensions to make the mask smaller
    #     new_x = int(x-0.1*x)  # Adjust the scaling factor as needed
    #     new_y = int(y - 0.3 * h)
    #     new_w = int(1.3* w)
    #     new_h = int(1.4*h)

    #     # Generate a mask for the adjusted face ROI
    #     mask[new_y:new_y+new_h, new_x:new_x+new_w] = 255

    # # Create an empty alpha channel
    # alpha = np.zeros_like(gray_image)

    # # Set the alpha channel to 255 (fully opaque) inside the face region
    # alpha[mask > 0] = 255

    # # Merge the RGB channels with the alpha channel
    # inverted_mask = 255 - mask

    # return inverted_mask

# if __name__ == "__main__":
#     test = resize_image("uploaded_image.png")
#     print(test.size)


    # cv2.imshow("test", test)
    # cv2.imwrite('hah.png', test)
    # cv2.waitKey(0)

    # list_img = test.flatten().tolist()

    # j_img = json.dumps(list_img)

    # list_img = json.loads(j_img)

    # image_array = np.array(list_img, dtype=np.uint8)

    # # Reshape the array to its original dimensions
    # original_height, original_width, channels = 1024, 1024, 3
    # image_array = image_array.reshape(original_height, original_width, channels)
    # cv2.imshow("image_array", image_array)
    # cv2.waitKey(0)



In [16]:
from pydantic import BaseModel

app = FastAPI()
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


class Item(BaseModel):
    text: str


@app.post("/upload/")
async def upload_image(file: UploadFile = File(...)):
    # Save the uploaded image locally
    with open("uploaded_image.png", "wb") as buffer:
        buffer.write(await file.read())

    resized_image = resize_image('uploaded_image.png')

    mask = create_mask(resized_image)

    # Check if image processing was successful
    if resized_image is None:
        return Response(status_code=500, content="Failed to process image")

    cv2.imwrite("resized_image.png", resized_image)

    cv2.imwrite("mask_image.png", mask)


    # # Return the processed image bytes with appropriate media type
    return 'Success'


@app.post("/process/")
async def process_image(item: Item):
    # Use the text parameter here
    transformed_image = transform_image('resized_image.png', 'mask_image.png', item.text)
    image_np = np.array(transformed_image)

    image_np = image_np.astype(np.uint8)

    cv2.imwrite('transformed.png', image_np)

    ready_image = image_np.flatten().tolist()

    # Convert list to JSON string
    transformed_image_json = json.dumps(ready_image)


    # Convert the transformed image to JSON format
    # transformed_image_json = json.dumps(transformed_image.flatten().tolist())

    return Response(content=transformed_image_json)


In [12]:
auth_token = "" #@param {type:"string"}
# Since we can't access Colab notebooks IP directly we'll use
# ngrok to create a public URL for the server via a tunnel

# Authenticate ngrok
# https://dashboard.ngrok.com/signup
# Then go to the "Your Authtoken" tab in the sidebar and copy the API key
import os
os.system(f"ngrok authtoken {auth_token}")



0

In [ ]:
from pyngrok import ngrok
import uvicorn
import nest_asyncio
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://f290-34-16-157-66.ngrok-free.app


INFO:     Started server process [342]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     178.51.245.47:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     178.51.245.47:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     178.51.245.47:0 - "POST /upload/ HTTP/1.1" 200 OK


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:     178.51.245.47:0 - "POST /process/ HTTP/1.1" 200 OK
INFO:     178.51.245.47:0 - "POST /upload/ HTTP/1.1" 200 OK


...

INFO:     178.51.245.47:0 - "POST /process/ HTTP/1.1" 200 OK
INFO:     178.51.245.47:0 - "POST /upload/ HTTP/1.1" 200 OK


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:     178.51.245.47:0 - "POST /process/ HTTP/1.1" 200 OK
INFO:     178.51.245.47:0 - "POST /upload/ HTTP/1.1" 200 OK


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:     178.51.245.47:0 - "POST /process/ HTTP/1.1" 200 OK
